<a href="https://colab.research.google.com/github/Gricay-vasily/project_9_DCGAN/blob/main/Learning_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Навчання Deep Convolutional Generative Adversarial Network (DCGAN) на завантажених даних Cifar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf

from tensorflow.keras import layers

## Завантаження і підготовка Дата-сетів

### Зчитування мета-даних

Посилання на дані

In [ ]:
URL_DIR = "https://www.cs.toronto.edu/~kriz/"
cifar=10

if cifar == 10:
    FILE_NAME_CIFAR = "cifar-10-python.tar.gz"
if cifar ==100:
    FILE_NAME_CIFAR = "cifar-100-python.tar.gz"

Завантажимо дані

In [ ]:
def load_and_unzip_cifar(url:str, file_name:str):
  '''Завантаження і розпакування файлу-архіву, якщо ще цього не зроблено'''
  from pathlib import Path
  # Завантаження даних
  full_url_to_load = url + file_name
  print(full_url_to_load)
  if Path(file_name).exists() and Path(file_name).is_file():
    print(f"Файл {file_name} вже є. Немає потреби його завантажувати!")
  else:
    print(f"Заватажуємо файл {file_name} ")
    !wget $full_url_to_load
  # Розпакування даних
  # Створення робочої папки через stem i split - відсікання суфіксів
  work_dir = Path(str((Path() / Path(file_name).stem)).split(".")[0])
  # print(work_dir)
  if work_dir.exists() and work_dir.is_dir():
    print(f"Директорія {work_dir} вже існує, перезаписуємо файли")
    # Перезаписуємо, бо є файли, які не дають видалити папку
    !tar -xzvf $file_name
  else:
    print(f"Розпакуємо файл {file_name}")
    !tar -xzvf $file_name

load_and_unzip_cifar(URL_DIR, FILE_NAME_CIFAR)

In [ ]:
def unpickle(file):
  '''Десеріалізація даних з pickle-файлу'''
  import pickle
  with open(file, 'rb') as fo:
      dict = pickle.load(fo, encoding='bytes')
  return dict

In [ ]:
if cifar == 10:
  metadata_path = './cifar-10-batches-py/batches.meta' # шлях до даних"
  metadata = unpickle(metadata_path)
  superclass_dict = dict(list(enumerate(metadata[b'label_names'])))
if cifar ==100:
  metadata_path = './cifar-100-python/meta' # шлях до даних
  metadata = unpickle(metadata_path)
  superclass_dict = dict(list(enumerate(metadata[b'coarse_label_names'])))

### Завантаження тренувальної та тестувальної вибірок (використовуючи суперкласи):

In [ ]:
if cifar == 100:
  data_pre_path = './cifar-100-python/' # шлях до даних
  # шляхи до файлів
  data_train_path = data_pre_path + 'train'
  data_test_path = data_pre_path + 'test'
  # Зчитуємо словники
  data_train_dict = unpickle(data_train_path)
  data_test_dict = unpickle(data_test_path)
  # Отримуємо дані (вибираємо coarse_labels щоб отримати всі 100 класів)
  data_train = data_train_dict[b'data']
  label_train = np.array(data_train_dict[b'coarse_labels'])
  data_test = data_test_dict[b'data']
  label_test = np.array(data_test_dict[b'coarse_labels'])

### Завантаження тренувальної та тестувальної вибірок для Cifar-10:

In [ ]:
if cifar == 10:
  data_pre_path = './cifar-10-batches-py/' # шлях до даних
  data_train = None
  label_train = []
  data_test = None
  label_test = []
  # Тренувальні вибірки
  for _ in range(1,6):
    data_train_dict = unpickle(data_pre_path + f"data_batch_{_}")
    if _ == 1:
      data_train = data_train_dict[b'data']
    else:
      data_train = np.vstack((data_train, data_train_dict[b'data']))
    label_train += data_train_dict[b'labels']
  #  Тестувальні вибірки
  data_test_dict = unpickle(data_pre_path + "test_batch")
  data_test = data_test_dict[b'data']
  label_test = data_test_dict[b'labels']

Поверхнево дослідимо дані

In [ ]:
type(data_train.shape), type(data_test.shape)

In [ ]:
data_train.shape, data_test.shape

In [ ]:
np.info(data_train)

In [ ]:
data_train.reshape(len(data_train), 3, 32, 32).shape, data_test.reshape(len(data_test), 3, 32, 32).shape

### Зміна розмірності зображень - виконувати лише раз у колабі!!!

In [ ]:
# Транспонується саме (0,2,3,1) - щоб отримати:
# 0 - Позицію картинки
# 2 - Значення висоти в пікселях
# 3 - Значення ширини в пікселях
# 1 - значення кольорів у RGB
# Якщо набрати (0,3,2,1) - картинка перевертається на 90град - висота і ширина міняються місцями

data_train = data_train.reshape(len(data_train), 3, 32, 32).transpose(0,2,3,1)
data_test = data_test.reshape(len(data_test), 3, 32, 32).transpose(0,2,3,1)

In [ ]:
np.info(data_train)

In [ ]:
_ = random.randint(0,len(data_train))
print(f"For picture #{_} - {label_train[_]} ({superclass_dict[label_train[_]]}):")
fig = plt.figure(figsize=(1,1))
fig.add_subplot(1,1,1)
plt.imshow(data_train[_])
plt.axis("off")
plt.show()

Трохи більше картинок з тренувальної та тестувальної вибірок...

In [ ]:
def show_pictures_n_m(data,n,m:int):
    fig = plt.figure(figsize=(max(n,m),max(n,m)))
    for _ in range (n*m):
        fig.add_subplot(n,m,_+1)
        plt.imshow(data[_])
        plt.axis("off")
    plt.show()

show_pictures_n_m(data_train,5,5)
show_pictures_n_m(data_test,3,3)

### Подальша підготовка даних для тренувань мережі

Перемішаємо тренувальні дані, використовуючи tensorflow

In [ ]:
data_train.shape[0]

In [ ]:
BUFFER_SIZE = data_train.shape[0]
BATCH_SIZE = 256

In [ ]:
def prepare_images(data_set, buffer_size, batch_size):
  prepared_images = data_set.astype("float32")
  prepared_images = (prepared_images - 255) / 255
  prepared_images = tf.data.Dataset.from_tensor_slices(prepared_images)
  #Перемішаємо дані випадковим чином
  prepared_images = prepared_images.shuffle(buffer_size=buffer_size).batch(batch_size=batch_size)
  return prepared_images

In [ ]:
train_images = prepare_images(data_train, BUFFER_SIZE, BATCH_SIZE)

In [ ]:
train_images

In [ ]:
len(train_images)

# Створення та тренування генератора і дискримінатора

Генератор - використовує шари `tf.keras.layers.Conv2DTranspose` (для підвищення дискретності),щоб створювати зображення з вихідного випадкового шуму (random noise). Починається шаром `Dense`, який приймає це початкове значення, а потім кілька разів підвищує дискретизацію, поки не досягне бажаного розміру зображення.

Дискримінатор - Це класифікатор зображень на основі CNN.

Він класифікує (визначає) якою є згенерована картинка - справжньою чи фейковою. Модель тренується показувати позитивне значення для справжньої картинки і негативне для фейкової.

In [ ]:
def real_or_fake(decision):
  if decision>=0:
    return "Real"
  else:
    return "Fake"

## Змінні

In [ ]:
# Рівень шуму
NOISE_DIM = 100
# Кількість генерованих зображень
NUM_EXAMPLES_TO_GENERATE = 16

## Функції створення генератора та дискримінатора

In [ ]:
def build_generator():
  """Створення генератора"""
  model = tf.keras.Sequential()
  model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(100,)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Reshape((8, 8, 256)))
  assert model.output_shape == (None, 8, 8, 256)

  model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
  assert model.output_shape == (None, 32, 32, 3)

  return model

def build_discriminator():
  """Створення дискримінатора"""
  model = tf.keras.Sequential()
  model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3]))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))

  return model

## Функції втрат для генератора і дискримінатора

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
  """Ф-я втрат генератора - через бінарну кросс-ентропію"""
  return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
  """Ф-я втрат дискримінатора - через бінарну кросс-ентропію"""
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  return real_loss + fake_loss

## Створення і налаштування генератора

In [ ]:
generator = build_generator()
generator.optimizer = tf.keras.optimizers.Adam(1e-4)

generator.summary()

## Створення і налаштування дискримінатора

In [ ]:
discriminator = build_discriminator()
discriminator.optimizer = tf.keras.optimizers.Adam(1e-4)

discriminator.summary()

In [ ]:
# Генерування зображення та його класифікація дискримінатором без навчання
noise = tf.random.normal([1, NOISE_DIM])
generated_image = generator(noise, training = False)
decision = discriminator(generated_image).numpy()[0][0]
print(f"generated_image.shape = {generated_image.shape}") #TensorShape([1, 32, 32, 3])
print(f"Точність decision = {decision}")
verdict = real_or_fake(decision)
print(f"Зображення - {verdict}")
plt.imshow(generated_image[generated_image.shape[0]-1, :,: ,0])
plt.show()

## Тренування моделей

In [ ]:
@tf.function
def gen_train_step(images, batch_size):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:

def preprocess_image(image):
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, [32, 32])
    image = (image - 127.5) / 127.5  # Normalize the images to [-1, 1]
    return image

def load_dataset(data_path):
    data = tf.keras.utils.get_file(data_path, origin=data_path)
    images = []  # List to store processed images
    for file in data:
        image = preprocess_image(tf.io.read_file(file))
        images.append(image)
    return tf.data.Dataset.from_tensor_slices(images).batch(32)
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)





generator = build_generator()
discriminator = build_discriminator()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        print(f'Epoch {epoch+1} completed')